In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import sys
import glob
import math
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import models
import torch.optim as optim
import torch.nn.functional as F
train_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/train15000/"
test_root = "/home/yulin/Documents/DLCV/hw2/hw2_train_val/val1500/"


In [3]:
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print('Device used:', device)

Device used: cpu


In [4]:
class MYDATASET(Dataset):
    def __init__(self, root, transform=None):
        """ Intialize the MNIST dataset """
        self.images = None
        self.labels = None
        self.filenames = []
        self.root = root
        self.transform = transform
        #     parent = os.path.join(path, i)　　read filenames
        path = root+"images/"
        path_ = root+"labelTxt_hbb/"
        self.filenames=sorted(glob.glob(os.path.join(path,'*.jpg')))
        self.txtnames=sorted(glob.glob(os.path.join(path_,'*.txt')))
        self.len = len(self.filenames)
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        image_fn = self.filenames[index]
        temp = Image.open(image_fn).convert('RGB')
        image = temp.resize((448, 448),Image.ANTIALIAS)    
        f = open(self.txtnames[index])
        line = f.readline()
        list = []
        while line:
            a = line.split( )
            b = a[0:]
            list.append(b)
            line = f.readline()
        f.close
        label_one=np.zeros((7,7,26))
        for s in range(len(list)):
            x=0.5*(float(list[s][2])+float(list[s][0]))/512
            y=0.5*(float(list[s][5])+float(list[s][1]))/512
            w=1*(float(list[s][2])-float(list[s][0]))/512
            h=1*(float(list[s][5])-float(list[s][1]))/512
            class_=list[s][8]
            diff_=list[s][9]
            i=math.floor(7*x)
            j=math.floor(7*y)
            if label_one[i][j][0]==0:
                
                label_one[i][j][0]=x
                label_one[i][j][1]=y
                label_one[i][j][2]=w
                label_one[i][j][3]=h
                label_one[i][j][4]=1
                if class_==str('plane'):
                    label_one[i][j][10]=1
                if class_==str('ship'):
                    label_one[i][j][11]=1
                if class_==str('storage-tank'):
                    label_one[i][j][12]=1
                if class_==str('baseball-diamond'):
                    label_one[i][j][13]=1
                if class_==str('tennis-court'):
                    label_one[i][j][14]=1
                if class_==str('basketball-court'):
                    label_one[i][j][15]=1
                if class_==str('ground-track-field'):
                    label_one[i][j][16]=1
                if class_==str('harbor'):
                    label_one[i][j][17]=1
                if class_==str('bridge'):
                    label_one[i][j][18]=1
                if class_==str('small-vehicle'):
                    label_one[i][j][19]=1
                if class_==str('large-vehicle'):
                    label_one[i][j][20]=1
                if class_==str('helicopter'):
                    label_one[i][j][21]=1
                if class_==str('roundabout'):
                    label_one[i][j][22]=1
                if class_==str('soccer-ball-field'):
                    label_one[i][j][23]=1
                if class_==str('swimming-pool'):
                    label_one[i][j][24]=1
                if class_==str('container-crane'):
                    label_one[i][j][25]=1
            else:
                pass
            label = torch.from_numpy(label_one).type(torch.float)
            

        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return self.len

In [5]:
trainset=MYDATASET(root=train_root, transform=transforms.ToTensor())
testset=MYDATASET(root=test_root, transform=transforms.ToTensor())

In [6]:
#print('# images in trainset:', len(trainset)) # Should print 15000
#print('# images in testset:', len(testset)) # Should print 1500

In [7]:
trainset_loader = DataLoader(trainset, batch_size=5, shuffle=True, num_workers=1)
testset_loader = DataLoader(testset, batch_size=5, shuffle=False, num_workers=1)
# get some random training images
dataiter = iter(trainset_loader)
images, labels = dataiter.next()

print('Image tensor in each batch:', images.shape, images.dtype)
print('Label tensor in each batch:', labels.shape, labels.dtype)

Image tensor in each batch: torch.Size([5, 3, 448, 448]) torch.float32
Label tensor in each batch: torch.Size([5, 7, 7, 26]) torch.float32


In [8]:
def judge(inputs):
    a=torch.unbind(inputs, dim=3)[1]
    index=torch.nonzero(a)
    return index

In [9]:
model = models.Yolov1_vgg16bn(pretrained=True)

In [10]:
ssd=model(images)
ssd.shape

torch.Size([5, 7, 7, 26])

In [11]:
index=judge(labels)

In [12]:
index=judge(labels)
def iou(inputs, targets, index, e):
    score=torch.zeros(len(index))
    a=0
    for k,i,j in index:
        squ_label=targets[k][i][j][0:4]
        squ_model=inputs[k][i][j][0+e:4+e]
        x_min1=squ_label[0]-0.5*squ_label[2]
        x_max1=squ_label[0]+0.5*squ_label[2]
        y_min1=squ_label[1]-0.5*squ_label[3]
        y_max1=squ_label[1]+0.5*squ_label[3]
        x_min2=squ_model[0]-0.5*squ_model[2]
        x_max2=squ_model[0]+0.5*squ_model[2]
        y_min2=squ_model[1]-0.5*squ_model[3]  
        y_max2=squ_model[1]+0.5*squ_model[3]

        ww=torch.min(x_max1,x_max2)-torch.max(x_min1,x_min2)
        hh=torch.min(y_max1,y_max2)-torch.max(x_min1,y_min2)
        b1_area=(x_max1-x_min1) *(y_max1-y_min1)
        b2_area=(x_max2-x_min2) *(y_max2-y_min2)

        if ww<=0 or hh<=0:
            a=a+1
        else:
            score[a] = (ww*hh) / (b1_area + b2_area - ww*hh + 1e-16)
            a=a+1

    return score
score=iou(ssd,labels,index,1)

print(score)

tensor([0.0284, 0.0000, 0.0000, 0.1473, 0.0000, 0.0000, 0.0000, 0.0000, 0.0079,
        0.1434], grad_fn=<CopySlices>)


In [13]:
class LossFunc(nn.Module):
    def __init__(self):
        super(LossFunc, self).__init__()   
        #number
        s=7
        ss=s*s
        coord=5
        noodj=0.5    
    
    def forward(self, inputs, targets):
        s=7
        ss=s*s
        coord=5
        noodj=0.5
        num=torch.numel(inputs[:, 0, 0, 0]) 
        index=judge(targets)
        delta_sum=0
        for k,i,j in index:
            temp=inputs[k, i, j, 1:3]-targets[k, i, j, 1:3]
            delta_xy1=torch.sum(torch.pow(temp,2))
            temp2=torch.sqrt(inputs[k, i, j, 3:5])-torch.sqrt(labels[k, i, j, 3:5])
            delta_wh1=torch.sum(torch.pow(temp2,2))
            temp3=inputs[k, i, j, 6:8]-targets[k, i, j, 1:3]
            delta_xy2=torch.sum(torch.pow(temp3,2))
            temp4=torch.sqrt(inputs[k, i, j, 8:10])-torch.sqrt(labels[k, i, j, 3:5])
            delta_wh2=torch.sum(torch.pow(temp4,2))
            delta_sum = delta_sum+delta_wh1+delta_xy1+delta_wh2+delta_xy2
        first=coord*delta_sum/num#中心點誤差
        
        confidence1=iou(inputs, targets, index,0)#獲得1置信度與概率
        confidence2=iou(inputs, targets, index,1)#獲得2置信度與概率
        pr=torch.le(confidence1, confidence2) 
        hat_c=torch.FloatTensor(len(confidence1),2).zero_()
        n1=torch.FloatTensor(len(confidence1)).zero_()
        n2=torch.FloatTensor(len(confidence1)).zero_()
        m1=torch.FloatTensor(len(confidence1)).zero_()
        m=0
        second=0
        for k,i,j in index:
                    hat_c[m][0]=inputs[k, i, j, 4:5]
                    hat_c[m][1]=inputs[k, i, j, 9:10]
                    m=m+1
        for t in range(len(confidence1)):
            if pr[t]==0:
                n1[t] = hat_c[t][0]
                n2[t] = hat_c[t][1]
                m1[t] = confidence1[t]
            else:
                n2[t] = hat_c[t][0]
                n1[t] = hat_c[t][1]
                m1[t] = confidence２[t]

        tar_1=torch.pow(n1-m1,2)
        tar_2=torch.pow(n2,2)
        second=torch.sum(tar_1+noodj*tar_2)

        third=0    
        for k,i,j in index:
            obj=torch.squeeze(targets[k, i, j, 10:])
            sss=torch.squeeze(inputs[k, i, j, 10:])
            e=torch.pow((obj-sss),2)
            third=third+torch.sum(e)
            
            
        loss=(first+second+third)/num
        return loss

In [147]:
def iou2(box1,box2):

    x_min1=box1[0]-0.5*box1[2]
    x_max1=box1[0]+0.5*box1[2]
    y_min1=box1[1]-0.5*box1[3]
    y_max1=box1[1]+0.5*box1[3]
    x_min2=box2[0]-0.5*box2[2]
    x_max2=box2[0]+0.5*box2[2]
    y_min2=box2[1]-0.5*box2[3]  
    y_max2=box2[1]+0.5*box2[3]
    ww=torch.min(x_max1,x_max2)-torch.max(x_min1,x_min2)
    hh=torch.min(y_max1,y_max2)-torch.max(y_min1,y_min2)
    b1_area=(x_max1-x_min1) *(y_max1-y_min1)
    b2_area=(x_max2-x_min2) *(y_max2-y_min2)

    interArea= max(0, hh) * max(0, ww)
    score = interArea / (b1_area + b2_area - ww*hh + 1e-16) 
    return score
for k,i,j in index:
    w=iou2(ssd[k][i][j][0:4],labels[k][i][j][0:4])
    print(w)

tensor(0.0263, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<MulBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0.0759, grad_fn=<DivBackward0>)
tensor(0.0785, grad_fn=<DivBackward0>)
tensor(0.0223, grad_fn=<DivBackward0>)
tensor(0., grad_fn=<DivBackward0>)
tensor(0., grad_fn=<MulBackward0>)
tensor(0.0021, grad_fn=<DivBackward0>)
tensor(0.2134, grad_fn=<DivBackward0>)


In [14]:
def train(model, epoch, log_interval=100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion=LossFunc()
    model.train()  # Important: set training mode
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1


In [98]:
train(model, 5)

Train Epoch: 0 [0/15000 (0%)]	Loss: 17.902967
Train Epoch: 0 [100/15000 (1%)]	Loss: 7.778686
Train Epoch: 0 [200/15000 (1%)]	Loss: 1.230989
Train Epoch: 0 [300/15000 (2%)]	Loss: 10.256804
Train Epoch: 0 [400/15000 (3%)]	Loss: 1.027899
Train Epoch: 0 [500/15000 (3%)]	Loss: 1.809370
Train Epoch: 0 [600/15000 (4%)]	Loss: 3.654349
Train Epoch: 0 [700/15000 (5%)]	Loss: 1.495774
Train Epoch: 0 [800/15000 (5%)]	Loss: 6.316463
Train Epoch: 0 [900/15000 (6%)]	Loss: 5.378819
Train Epoch: 0 [1000/15000 (7%)]	Loss: 0.897084
Train Epoch: 0 [1100/15000 (7%)]	Loss: 3.616435
Train Epoch: 0 [1200/15000 (8%)]	Loss: 1.577830
Train Epoch: 0 [1300/15000 (9%)]	Loss: 4.858388
Train Epoch: 0 [1400/15000 (9%)]	Loss: 9.453559
Train Epoch: 0 [1500/15000 (10%)]	Loss: 41.105217
Train Epoch: 0 [1600/15000 (11%)]	Loss: 3.864344
Train Epoch: 0 [1700/15000 (11%)]	Loss: 0.752075
Train Epoch: 0 [1800/15000 (12%)]	Loss: 1.732554
Train Epoch: 0 [1900/15000 (13%)]	Loss: 33.614349
Train Epoch: 0 [2000/15000 (13%)]	Loss: 17.

Train Epoch: 1 [1700/15000 (11%)]	Loss: 2.843520
Train Epoch: 1 [1800/15000 (12%)]	Loss: 11.217308
Train Epoch: 1 [1900/15000 (13%)]	Loss: 0.491225
Train Epoch: 1 [2000/15000 (13%)]	Loss: 0.685102
Train Epoch: 1 [2100/15000 (14%)]	Loss: 5.427510
Train Epoch: 1 [2200/15000 (15%)]	Loss: 16.527828
Train Epoch: 1 [2300/15000 (15%)]	Loss: 10.634339
Train Epoch: 1 [2400/15000 (16%)]	Loss: 3.207422
Train Epoch: 1 [2500/15000 (17%)]	Loss: 4.747562
Train Epoch: 1 [2600/15000 (17%)]	Loss: 1.056015
Train Epoch: 1 [2700/15000 (18%)]	Loss: 3.316047
Train Epoch: 1 [2800/15000 (19%)]	Loss: 13.803205
Train Epoch: 1 [2900/15000 (19%)]	Loss: 18.562969
Train Epoch: 1 [3000/15000 (20%)]	Loss: 12.270779
Train Epoch: 1 [3100/15000 (21%)]	Loss: 16.378767
Train Epoch: 1 [3200/15000 (21%)]	Loss: 0.488505
Train Epoch: 1 [3300/15000 (22%)]	Loss: 0.528304
Train Epoch: 1 [3400/15000 (23%)]	Loss: 6.785482
Train Epoch: 1 [3500/15000 (23%)]	Loss: 7.996471
Train Epoch: 1 [3600/15000 (24%)]	Loss: 0.752838
Train Epoch: 

Traceback (most recent call last):
  File "/home/yulin/anaconda3/envs/dlcv/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/yulin/anaconda3/envs/dlcv/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/yulin/anaconda3/envs/dlcv/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/yulin/anaconda3/envs/dlcv/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [384]:
inputs=ssd
def get_score(inputs):
    tem=inputs[:,0,0,0]
    #for p in range(torch.numel(tem)):
    boxobject,ind=torch.max(inputs[:,:,:,10:],3)#max class score
    point2=inputs[:,:,:,9]
    point1=inputs[:,:,:,4]
    m = nn.Threshold(0.1, 0)
    k=torch.cat((point2,point1) ,1)
    t=torch.cat((boxobject,boxobject) ,1)
    u=torch.cat((ind,ind) ,1)
    kt=m(k*t) #置信度分數

    get_index=torch.LongTensor()
    lists=torch.LongTensor()
    score=torch.ones(kt.size())

    while torch.max(score) != 0:

        for p in range(torch.numel(tem)):

            s=torch.argmax(kt[p,:])
            d=torch.max(kt[p,:])
            y=s%7;
            x=((s-y)/7)%7
            z=ind[p][x%7][y]
            k=x/7

            kt[p,x,y]=0

            lists=torch.cat((torch.tensor([p]),x.view(1),y.view(1),z.view(1)),0).view(1,-1)
            get_index=torch.cat((get_index,lists),0)

            for p in range(torch.numel(tem)):
                for i in range(7):
                    for j in range(7):
                        for u in range(2):
                            box1=inputs[p,i,j,0+u:4+u]
                            box2=inputs[p,x,y,0+k:4+k]
                            iou = iou2(box1,box2)
                            if iou > 0.5:
                                score[p][i+u*7][j]=0
                            else:
                                pass

    return get_index

In [383]:
get_score(inputs)

tensor([[ 0,  0,  1,  0],
        [ 1,  3,  0, 15],
        [ 2,  6,  6,  3],
        [ 3,  6,  0, 10],
        [ 4,  2,  5,  2],
        [ 0,  5,  6, 11],
        [ 1,  3,  0, 15],
        [ 2,  1,  4,  8],
        [ 3,  2,  3,  1],
        [ 4,  2,  5,  2],
        [ 0,  5,  6, 11],
        [ 1,  3,  0, 15],
        [ 2,  2,  3,  1],
        [ 3,  2,  3,  1],
        [ 4,  2,  5,  2],
        [ 0,  5,  6, 11],
        [ 1,  3,  0, 15],
        [ 2,  0,  2, 12],
        [ 3,  2,  3,  1],
        [ 4,  2,  5,  2]])

In [367]:
kt.size()

torch.Size([5, 14, 7])

In [180]:
    x,y=model_max(inputs)
    max_result=torch.FloatTensor(len(x),4).zero_()
    score=torch.FloatTensor(len(x),2*7,7).zero_()
    for q in range(5):
        if x[q]>=7:
            max_result[q,:]=inputs[q][x[q]-7][y[q]][5:9]
        else:
            max_result[q,:]=inputs[q][x[q]][y[q]][0:4]
        b=0
        for i in range(7):
            for j in range(7):
                squ_label=inputs[q][i][j][0:4]
                squ_model=max_result[q,:]

print(score)

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [16]:
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [17]:
def train_save(model, epoch, save_interval, log_interval=100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion=LossFunc()
    model.train()
    
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            if iteration % save_interval == 0 and iteration > 0:
                save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)
            iteration += 1
    
    # save the final model
    save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)

In [ ]:
# create a brand new model
model = Net().to(device)
test(model)
train_save(model, 5, 500, 100)